In [13]:
import gym 
import numpy as np 
env=gym.make('Taxi-v3')
env.render() # 환경을 화면에 출력 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [14]:
#환경 초기화
env.reset()
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [15]:
print('Action Space:',env.action_space) # 6가지 동작
print('State Space:', env.observation_space) # 500가지 경우의 수 

Action Space: Discrete(6)
State Space: Discrete(500)


In [16]:
# 택시의 행, 택시의 열, 승객위치, 목적지
state = env.encode(3,1,2,3)
print(state)
env.s = state
env.render()

331
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [17]:
env.P[331] # 현재 위치에서의 초기 보상 테이블

{0: [(1.0, 431, -1, False)],
 1: [(1.0, 231, -1, False)],
 2: [(1.0, 351, -1, False)],
 3: [(1.0, 331, -1, False)],
 4: [(1.0, 331, -10, False)],
 5: [(1.0, 331, -10, False)]}

In [18]:
#초기 상태
env.s =328 
epochs=0
#패널티, 보상
penalties, reward=0, 0
#애니메이션을 위하여 저장할 프레임의 리스트
frames=[]
#게임 종료 여부
done=False 
while not done:
    #랜덤으로 동작 선택
    action=env.action_space.sample()
    #동작을 취하고 결과값 저장
    state,reward,done,info=env.step(action)
    #보상이 -10이면 패널티 1 추가
    if reward==-10:
        penalties += 1
    #애니메이션을 위한 프레임 저장
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state, 
        'action': action, 
        'reward': reward 
    })
    epochs+=1

print('Timesteps taken:{}'.format(epochs))    
print('Penalties incurred:{}'.format(penalties))

Timesteps taken:200
Penalties incurred:72


In [19]:
from IPython.display import clear_output 
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        #화면을 지우고
        clear_output(wait=True)
        #프레임을 출력
        print(frame['frame'])
        print(f'Timestep: {i+1}')
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        #0.1초 멈춤
        sleep(0.1)

print_frames(frames)         


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)

Timestep: 200
State: 363
Action: 4
Reward: -10


In [20]:
# Q테이블 초기화 : 작업공간의 경우의 수 x 동작의 경우의 수
q_table=np.zeros([env.observation_space.n, env.action_space.n])
print(q_table.shape )

(500, 6)


In [21]:
%%time 
import random 
alpha=0.1 #학습률
gamma=0.6 #할인율
epsilon=0.1

all_epochs=[]
all_penalties=[]
for i in range(1,100001):
    #환경 초기화
    state=env.reset()
    epochs,penalties,reward = 0,0,0 
    done=False 
    while not done:
        # epsilon에 지정한 비율만큼 랜덤으로 탐색
        if random.uniform(0,1) < epsilon:   
            action=env.action_space.sample()
        else: # Q 테이블의 값에 기반하여 행동 선택
            action=np.argmax(q_table[state])
        # 이동
        next_state,reward,done,info=env.step(action)
        # 현재 상태의 보상
        old_value=q_table[state, action]
        # 미래의 최대 보상
        next_max = np.max(q_table[next_state]) 
        # 새로운 보상값
        new_value=(1-alpha) * old_value + alpha*(reward+gamma+next_max)
        # Q 테이블 업데이트
        q_table[state, action]=new_value 
        # 보상이 -10이면 패널티 증가
        if reward == -10:
            penalties += 1
        state= next_state 
        epochs += 1 
    if i % 100 == 0 :
        clear_output(wait=True)
        print(f'Episode:{i}')
print('Training finished.\n')        

Episode:100000
Training finished.

CPU times: user 1min 3s, sys: 10.6 s, total: 1min 14s
Wall time: 1min 5s


In [22]:
# 학습이 완료된 328 위치의 Q 테이블, 각 동작에 대한 보상값 
q_table[328]

array([11.83656634, 17.        , 14.40578309, 12.00600229,  6.03402377,
        6.51068194])

In [23]:
#학습이 완료된 후 Q 테이블을 기반으로 움직이는 게임 
total_epochs, total_penalties=0,0
episodes=100
frames=[] 
for _ in range(episodes):
    state=env.reset()
    epochs, penalties, reward = 0,0,0 
    done=False 
    while not done:
        #현재 상태에서의 최대 보상값
        action=np.argmax(q_table[state])
        #이동
        state, reward, done, info=env.step(action)
        if reward== -10:
            penalties += 1 
        #애니메이션을 위한 프레임 저장
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state, 
            'action': action, 
            'reward': reward 
        })
        epochs+=1
    total_penalties += penalties
    total_epochs += epochs

print(f'Results after {episodes} episodes:')    
print(f'에피소드당 평균 시간 간격:{total_epochs / episodes}')
print(f'에피소드당 평균 패널티:{total_penalties / episodes}')

Results after 100 episodes:
에피소드당 평균 시간 간격:13.51
에피소드당 평균 패널티:0.0


In [24]:
print_frames(frames) 

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1351
State: 85
Action: 5
Reward: 20
